In [224]:
#導入相關模組
import json
import random
import re

import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

#讀取jieba所需的自建辭典
jieba.load_userdict("E:/Python 3.7/pyetl/Demodb0103/gym/data/dict.txt")
import time

In [ ]:
#讀取要斷詞的json檔
DATASET_DIR = 'E:/Python 3.7/pyetl/Demodb0103/gym/data/gymz.json'
with open(DATASET_DIR, encoding='utf8') as f:
    dataset = json.load(f)

In [225]:
#讀取停止詞典
STOP_WORDS_DIR = 'E:/Python 3.7/pyetl/Demodb0103/gym/stopWords.txt'
with open(STOP_WORDS_DIR, encoding='utf8') as f:
    stop_words = f.read().splitlines() 

In [226]:
#抽出文章的標題與內容
content_list = list(map(lambda d: d['content'], dataset))
title_list = list(map(lambda d: d['title'], dataset))

In [228]:
#替換不需要的句子，並正規化
# start = time.clock()
gym_list =[]
for i in content_list:
    i = re.sub('※ 發信站: 批踢踢實業坊',' ', i)
    i = re.sub('※ 文章網址:',' ', i)
    i = re.sub('※ 編輯:',' ', i)
    i = re.sub('(臺灣)',' ', i)
    i = re.sub('來自:',' ', i)
    #只保留中文
    rule = re.compile(r"[^\u4e00-\u9fa5]")
    i = rule.sub('', i)
    gym_list.append(i)


In [229]:
#將文章正規化，並進行jieba斷詞
rule = re.compile(r"[^\u4e00-\u9fa5]")
gym_list = [list(jieba.cut(rule.sub('', speech))) for speech in gym_list1]

In [230]:
##去除停止詞
for idx, speech in enumerate(gym_list):
    gym_list[idx] = ' '.join([word for word in speech if word not in stop_words])
# end = time.clock()
# print('運行時間: ' + str(end - start))

In [284]:
#Word2Vec
# 資料形式為一個文檔各為list元素為字串
from gensim.models import Word2Vec

#全文章轉為各list包起來
doc_clean1 = [doc.split() for doc in word_s]

# 建立模型
#window:CBOW下決定Word2Vec一次取多少詞來預測中間詞
#min_count:出現次數大於等於min_count的詞，才會納入Word2Vec的詞典中
#negative:Negative Sampling的取樣數量，5~20適合小數據，2~5適合大數據
# worker=使用多核計算機進行更快的訓練
model = Word2Vec(doc_clean1,window=5, negative=5,min_count=1)

# # 基於2d PCA擬合數據
# X = model[model.wv.vocab]
# pca = PCA(n_components=2)
# result = pca.fit_transform(X)

E:\Python 3.7\pyetl\Demodb0103\venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [277]:
#顯示指定詞的關聯詞
model2.wv.most_similar('肥')

[('瘦子', 0.9213787913322449),
 ('胖子', 0.9148010015487671),
 ('瘦胖子', 0.9103908538818359),
 ('腿頸', 0.9088810086250305),
 ('瘦子練壯', 0.9006045460700989),
 ('死胖子', 0.890883207321167),
 ('胖', 0.8893124461174011),
 ('泡芙出', 0.8852933645248413),
 ('泡芙', 0.8737491369247437),
 ('肥宅臉', 0.872066855430603)]

In [271]:
#顯示各個指定詞的關聯詞
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df
most_similar(model, ['增肌', '肥'])

,增肌,cos,肥,cos
0,減脂,0.931740,瘦子,0.943572
1,肪,0.907604,胖子,0.932490
2,期,0.878710,胖,0.898200
3,增脂,0.819122,瘦下來,0.894688
4,增到,0.818283,肉肉,0.880289
5,壯起來,0.804180,變腫,0.873781
6,增重,0.795238,紙片人,0.871549
7,爬增,0.787592,瘦瘦,0.869552
8,拖垮,0.778630,身形,0.867059
9,小琉球,0.777449,復,0.861484


In [265]:
# import word2vec #需要word2vec模組
# indexes = model.cosine(u'增肌')
# for index in indexes[0]:
#     print (model.vocab[index])

ModuleNotFoundError: No module named 'word2vec'

In [275]:
# fasttext
from gensim.models import FastText
model2 = FastText(doc_clean1,window=5, negative=5,min_count=1)
